# Permutations

This notebook demonstrates the effect of changing different settings.

In [ ]:
image_model
- vqgan - ...

perceptor
- ...
- ...

reencode each frame

##############

# image_model fixed

animation
preanimation
camera lock

cutouts

cutpow

stabilization modes

border mode

sampling mode

infill mode

#############################

palettes

palette size

smoothing

gamma

hdr weight

palette normalization

lock palette

target palette

+/- stabilization weights, modes, etc.

#############################




In [ ]:
python -m pytti.workhorse --multirun \
python -m pytti.workhorse \
  allow_overwrite=false \
  height=256 width=512 \
  file_namespace=permutations_limited_palette_2D \
  scenes="fractal crystals" \
  image_model="Limited Palette" \
  palettes=5,10,30,70 \
  palette_size=3,7,15,30 \
  steps_per_frame=50 \
  steps_per_scene=1000 \
  cutouts=20,60,80 \
  cut_pow=0.5,1,1.5,2 \
  animation_mode=2D \
  translate_y=-10 \
  zoom_x_2d=5 \
  zoom_y_2d=5


In [ ]:
python -m pytti.workhorse \
  file_namespace=permutations_limited_palette_2D \
  scenes="fractal crystals" \
  image_model="Limited Palette" \
  palettes=5


  palette_size=3,7,15,30 \
  steps_per_frame=50 \
  steps_per_scene=1000 \
  cutouts=20,60,80 \
  cut_pow=0.5,1,1.5,2 \
  animation_mode=2D \
  translate_y=-10 \
  zoom_x_2d=15 \
  zoom_y_2d=15 \

In [ ]:
%%capture
%matplotlib inline
from pittybook_utils import (
    ExperimentMatrix
)

exp = ExperimentMatrix(
    variant = dict(
        palettes=(10,30,70),
        palette_size=(3,7,15),
        #cutouts=(10,50,100),
        #cut_pow=(0.5,1,1.5,2),
        gamma=(0, 0.1, 1),
        hdr_weight=(0, 0.1, 1),
        smoothing_weight=(0, 0.1, 1),
        #lock_palette=(True,False),
        palette_normalization_weight=(0, 0.1, 1),
    ),
    invariant = dict(
        lock_palette=False,
        cutouts=60,
        cut_pow=1,
        allow_overwrite=False,
        pixel_size=1,
        height=128,
        width=256,
        #file_namespace="permutations_limited_palette_2D",
        scenes="fractal crystals | colorful recursions || swirling curves | ethereal neon glow ",
        scene_suffix=" | text:-1:-.9 | watermark:-1:-.9",
        image_model="Limited Palette",
        steps_per_frame=50,
        steps_per_scene=1000,
        interpolation_steps=500,
        animation_mode="2D",
        translate_y=-1,
        zoom_x_2d=3,
        zoom_y_2d=3,
        seed=12345,
    ),
    # variable imputation doesn't seem to work in the overrides
    mapped = {
        'steps_per_frame':('pre_animation_steps', 'save_every'),
        'steps_per_scene':('display_every',),
    },
    #conditional = {'gradient_accumulation_steps': lambda kws: 1 if kws['cutouts'] < 100 else 4}
    conditional = {'file_namespace': 
                    lambda kws: '_'.join(
                        ["permutations_limited_palette_2D"]+[
                            f"{k}-{v}" for k,v in kws.items() if k in ('palettes','palette_size','gamma','hdr_weight','smoothing_weight','palette_normalization_weight')]
                            )},
)



In [ ]:
%%time 
exp.run_all()